In [6]:
import optuna

from optuna.trial import Trial

from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [11]:
models = {
    "SVC": SVC,
    "DecisionTree": DecisionTreeClassifier,
}

params = {
    "SVC": {
        'C': ['log', 1e-10, 1e10],
        'gamma': ['log', 1e-10, 1e1],
    },
    "DecisionTree": {
        'max_depth': ['int', 1, 20],
        'min_samples_split': ['int', 2, 10],
    },
}

In [15]:
def trial_type(trial: Trial, name: str, config: dict):
    TYPE, START, END = 0, 1, 2
    ptype = config[TYPE]
    if ptype == 'log':     return trial.suggest_float(name, config[START], config[END], log=True)
    if ptype == 'int':     return trial.suggest_int(name, config[START], config[END])
    if ptype == 'flt':     return trial.suggest_float(name, config[START], config[END])
    if ptype == 'cat':     return trial.suggest_categorical(name, config[START:])
    raise ValueError(f"Unknown trial type: {trial_type}")

def get_trial_params(trial: Trial, model_name: str):
    model_params = {}
    for param_name, param_range in params[model_name].items():
        suggest_fn = trial_type(trial, param_name, param_range)
        model_params[param_name] = suggest_fn
    return model_params

In [16]:
iris = load_iris()
X, y = iris.data, iris.target


from sklearn.model_selection import cross_val_score

def objective(trial: Trial, model_name: str, X, y):
    model_cls = models[model_name]
    trial_params = get_trial_params(trial, model_name)

    # Instantiate model with trial params
    model = model_cls(**trial_params)
    
    # Evaluate using cross-validation
    score = cross_val_score(model, X, y, cv=3, scoring='accuracy').mean()
    return score

for model_name in models.keys():
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, model_name, X, y), n_trials=20)
    print(f"Best params for {model_name}: {study.best_params}")


[I 2025-10-23 19:57:22,443] A new study created in memory with name: no-name-80b395d0-bac3-4425-9168-31bc6d92ff8b
[I 2025-10-23 19:57:22,461] Trial 0 finished with value: 0.32 and parameters: {'C': 7.506572389991047e-10, 'gamma': 1.8074410723618524e-09}. Best is trial 0 with value: 0.32.
[I 2025-10-23 19:57:22,488] Trial 1 finished with value: 0.32 and parameters: {'C': 1.1540080767307265e-08, 'gamma': 9.603977853815309}. Best is trial 0 with value: 0.32.
[I 2025-10-23 19:57:22,502] Trial 2 finished with value: 0.32 and parameters: {'C': 0.00015634730893542928, 'gamma': 0.269873023214578}. Best is trial 0 with value: 0.32.
[I 2025-10-23 19:57:22,512] Trial 3 finished with value: 0.9666666666666667 and parameters: {'C': 427070365.31741804, 'gamma': 1.9975003137319836}. Best is trial 3 with value: 0.9666666666666667.
[I 2025-10-23 19:57:22,524] Trial 4 finished with value: 0.9866666666666667 and parameters: {'C': 222360.6191643117, 'gamma': 4.042949338272375e-06}. Best is trial 4 with va

Best params for SVC: {'C': 222360.6191643117, 'gamma': 4.042949338272375e-06}


[I 2025-10-23 19:57:23,030] Trial 15 finished with value: 0.9733333333333333 and parameters: {'max_depth': 3, 'min_samples_split': 3}. Best is trial 3 with value: 0.9733333333333333.
[I 2025-10-23 19:57:23,043] Trial 16 finished with value: 0.9666666666666667 and parameters: {'max_depth': 10, 'min_samples_split': 8}. Best is trial 3 with value: 0.9733333333333333.
[I 2025-10-23 19:57:23,058] Trial 17 finished with value: 0.9666666666666667 and parameters: {'max_depth': 4, 'min_samples_split': 4}. Best is trial 3 with value: 0.9733333333333333.
[I 2025-10-23 19:57:23,072] Trial 18 finished with value: 0.9666666666666667 and parameters: {'max_depth': 7, 'min_samples_split': 8}. Best is trial 3 with value: 0.9733333333333333.
[I 2025-10-23 19:57:23,084] Trial 19 finished with value: 0.9666666666666667 and parameters: {'max_depth': 15, 'min_samples_split': 5}. Best is trial 3 with value: 0.9733333333333333.


Best params for DecisionTree: {'max_depth': 7, 'min_samples_split': 3}
